In [1]:
import requests
import pandas as pd
import glob

In [3]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("CSV_Categories_clean/*.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(subset=['video_id'], keep="first", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1929 entries, 0 to 1947
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   video_id               1929 non-null   object 
 1   title                  1929 non-null   object 
 2   description            998 non-null    object 
 3   channel_id             1929 non-null   object 
 4   category_id            1929 non-null   int64  
 5   language               1929 non-null   object 
 6   channel                1929 non-null   object 
 7   published_at           1929 non-null   object 
 8   duration               1928 non-null   object 
 9   views                  1929 non-null   int64  
 10  likes                  1929 non-null   int64  
 11  comments               1929 non-null   int64  
 12  Engagement total       1929 non-null   int64  
 13  Taux d'engagement (%)  1929 non-null   float64
 14  duration_td            1928 non-null   object 
 15  Durée (s)

In [5]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [7]:
#supprimer autres langues
df['language'].value_counts()

language
fr         1066
en          425
hi          251
de           21
es           20
id           15
und          14
zxx          13
ta           12
ar           10
ru            9
ur            7
tr            7
bn            6
or            6
vi            5
pt-PT         5
pl            4
bn-IN         4
de-DE         4
it            2
uz            2
pt-BR         2
te            2
nl-NL         2
ko            2
ne            1
ml            1
hi-Latn       1
zh            1
nl            1
zh-CN         1
gu            1
es-MX         1
ja            1
chr           1
wo            1
pa            1
cs            1
Name: count, dtype: int64

In [9]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [11]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse
import re

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')

# OPTIONNEL : Si vous voulez remplacer les vides par 0 et avoir des entiers classiques
# df['Durée (s)'] = df['duration_td'].dt.total_seconds().fillna(0).astype(int)

print(df[['duration', 'Durée (s)']])

      duration  Durée (s)
0        PT11S         11
1        PT26S         26
2     PT10M29S        629
3        PT58S         58
4        PT17S         17
...        ...        ...
1943    PT1M3S         63
1944   PT2M39S        159
1945     PT15S         15
1946     PT59S         59
1947     PT53S         53

[1929 rows x 2 columns]


In [13]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id                   0
title                      0
description              931
channel_id                 0
category_id                0
language                   0
channel                    0
published_at               0
duration                   1
views                      0
likes                      0
comments                   0
Engagement total           0
Taux d'engagement (%)      0
duration_td                1
Durée (s)                  1
dtype: int64

In [15]:
df.shape

(1929, 16)